In [1]:
!pip install transformers
!pip install openai-clip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.1 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368607 sha256=19c0fc218453db8033b644e8265a96cab19f914d7b8f2aa3133eb518a91a1f93
  Stored in directory: /root/.cache/pip/wheels/08/77/8e/8d2f862df6bf7fb4e2007062d2cbaeae49862ec7b56d041229
Successfully built openai-clip


In [ ]:
import random
import numpy as np

phrase_list = ['2d', 'pixel art', 'cave', 'scifi', 'side scrolling', 'chibi', 'waifu', 'space ship', 'desert', 'city', 'wasteland', 'mega structure', 'steal', 'stone', 'rock']

def prompt_generator(phrase_list, prompt_word_length=32):
    prompt = ''
    while len(prompt) < prompt_word_length:
        phrase = random.choice(phrase_list)
        if len(prompt) + len(phrase) + 1 <= prompt_word_length:
            prompt += phrase + ' '
        else:
            break
    return prompt.strip()

# example usage:
prompt=prompt_generator(phrase_list) 
print(prompt_generator(phrase_list))


wasteland 2d pixel art cave


In [ ]:
print(prompt)

scifi mega structure rock cave


In [ ]:
import torch
import clip
import numpy as np

use_cuda = torch.cuda.is_available()
if use_cuda:
    # Move tensors to CUDA device
    device = torch.device('cuda:0')
    print(f"Using CUDA device: {torch.cuda.get_device_name(device)}")
else:
    # Use CPU
    device = torch.device('cpu')
    print("Warning: CUDA not available, using CPU.")

# Load the CLIP model
model, preprocess = clip.load("ViT-L/14", device=device)

# Tokenize and encode the prompt
prompt = "scifi mega structure rock cave"
tokens = clip.tokenize(prompt).to(device)
print("Size of tokens:", tokens.shape)
with torch.no_grad():
    text_features = model.encode_text(tokens).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)

# Convert the embedding to a numpy array
embedding = text_features.cpu().numpy()

print("Text embedding shape:",embedding.shape)  # should print (1, 768)




100%|███████████████████████████████████████| 890M/890M [00:15<00:00, 61.7MiB/s]


Size of tokens: torch.Size([1, 77])
Text embedding shape: (1, 768)


Run for obtaining random prompts, word embeddings and clip embeddings

In [2]:
import json
import torch
import clip
import random
import numpy as np
from typing import List
from torch import nn
from transformers import CLIPTokenizer, CLIPTextModel

phrase_list = ['2d', 'pixel art', 'cave', 'scifi', 'side scrolling', 'chibi', 'waifu', 'space ship', 'desert', 'city', 'wasteland', 'mega structure', 'steal', 'stone', 'rock']

N=500

# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14", device=device)



class CLIPTextEmbedder(nn.Module):
    """
    ## CLIP Text Embedder
    """

    def __init__(self, version: str = "openai/clip-vit-large-patch14", device="cuda:0", max_length: int = 12):
        """
        :param version: is the model version
        :param device: is the device
        :param max_length: is the max length of the tokenized prompt
        """
        super().__init__()
        # Load the tokenizer
        self.tokenizer = CLIPTokenizer.from_pretrained(version)
        print(self.tokenizer)
        # Load the CLIP transformer
        self.transformer = CLIPTextModel.from_pretrained(version).eval()

        #self.device = device
        self.device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
  
        self.max_length = max_length

    def forward(self, prompts: List[str]):
        """
        :param prompts: are the list of prompts to embed
        """
        # Tokenize the prompts
        batch_encoding = self.tokenizer(prompts, truncation=True, max_length=self.max_length, return_length=True,
                                        return_overflowing_tokens=False, padding="max_length", return_tensors="pt")
        # Get token ids
        tokens = batch_encoding["input_ids"].to(self.device)
        # Get CLIP embeddings
        return self.transformer(input_ids=tokens).last_hidden_state

def generate_prompts(num_prompts, phrase_list, prompt_word_length,x):
    prompts = []
    for i in range(num_prompts):
        # Generate prompt
        prompt = ''
        while len(prompt) < prompt_word_length:
            phrase = random.choice(phrase_list)
            if len(prompt) + len(phrase) + 1 <= prompt_word_length:
                prompt += phrase + ' '
            else:
                break
        prompt = prompt.strip()
        
        # Get word embedding
        with torch.no_grad():
            tokens = clip.tokenize(prompt).to(device)
            print(tokens.shape)
            print(tokens)
            print(type(tokens))
            out = x.forward(prompt)
            out=torch.flatten(out)
            out=torch.unsqueeze(out,0)
            print(out)
            print(type(out))
            print(out.shape)
        #    text_features = model.encode_text(tokens).float()
            print('Computing text embedding for Prompt'+str(i+1))
            text_features = model.encode_text(tokens).float()
            text_features /= text_features.norm(dim=-1, keepdim=True)
            print(text_features.shape)
            prompt_data = {
            "prompt": prompt,
            "word_embedding": out[0].tolist(),
            "clip_embedding": text_features[0].tolist(),
        }

        # Append prompt, word embedding, and clip embedding to list
        prompts.append(prompt_data)    
    # Save prompts as JSON to file
    with open("prompts_and_embeddings.json", "w") as f:
      json.dump(prompts, f)

x = CLIPTextEmbedder()
prompt_word_length=32
generate_prompts(N, phrase_list,prompt_word_length,x)
print('Process completed and dataset is generated')

100%|████████████████████████████████████████| 890M/890M [00:06<00:00, 141MiB/s]


CLIPTokenizer(name_or_path='openai/clip-vit-large-patch14', vocab_size=49408, model_max_length=77, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.21.layer_norm2.bias', 'vision_model.encoder.layers.11.self_attn.v_proj.weight', 'vision_model.encoder.layers.4.self_attn.out_proj.bias', 'vision_model.encoder.layers.21.mlp.fc1.weight', 'vision_model.encoder.layers.19.mlp.fc1.bias', 'vision_model.encoder.layers.15.layer_norm2.weight', 'vision_model.encoder.layers.4.mlp.fc1.weight', 'vision_model.encoder.layers.2.mlp.fc2.bias', 'vision_model.encoder.layers.15.self_attn.out_proj.bias', 'vision_model.encoder.layers.11.layer_norm2.bias', 'vision_model.encoder.layers.5.self_attn.k_proj.weight', 'vision_model.encoder.layers.10.self_attn.v_proj.weight', 'vision_model.encoder.layers.15.self_attn.q_proj.weight', 'vision_model.encoder.layers.16.mlp.fc2.weight', 'vision_model.encoder.layers.4.self_attn.q_proj.weight', 'vision_model.encoder.layers.18.self_attn.v_proj.bias', 'vision_model.encoder.layers.

Streaming output truncated to the last 5000 lines.
Computing text embedding for Prompt167
torch.Size([1, 768])
torch.Size([1, 77])
tensor([[49406,   273,   323,  9068,  5285, 13241,   794, 49407,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]], dtype=torch.int32)
<class 'torch.Tensor'>
tensor([[-0.3884,  0.0229, -0.0522,  ...,  0.2657, -0.8798, -1.3760]])
<class 'torch.Tensor'>
torch.Size([1, 9216])
Computing text embedding for Prompt168
torch.Size([1, 768])
torch.Size([1, 77])
tensor([[49406, 44

In [ ]:
import torch

# Declare a variable
a = torch.tensor([2, 3, 4, 5])
# Using unsqueeze() method to add dimension
print(a.shape)
m=torch.unsqueeze(a, 0)
# Print output
print(m.shape)

torch.Size([4])
torch.Size([1, 4])


In [ ]:
!pwd

/content


Elm auto-encoder implementation

In [3]:
import time
import torch
import numpy as np
import torch.nn as nn
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity


class ELMAutoEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size=1, use_gpu=True):
        super(ELMAutoEncoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
        self.use_gpu = use_gpu
        if self.use_gpu and torch.cuda.is_available():
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")
            print("Warning: CUDA not available, using CPU.")
        self.set_random_seed()    
        self.weight = nn.Parameter(torch.randn(input_size, hidden_size, dtype=torch.float, device=self.device))
        self.bias = nn.Parameter(torch.randn(hidden_size, device=self.device))
        self.beta = nn.Parameter(torch.randn(hidden_size, output_size, device=self.device))
        self.activation_hidden = nn.ReLU()
        self.activation_output = nn.Identity()
        
    def forward(self, x):
        h = self.activation_hidden(x @ self.weight + self.bias)       
        y_pred = torch.sigmoid(h @ self.beta)  # apply sigmoid to output
        return y_pred

    def set_random_seed(self):
        seed = int(time.time())
        torch.manual_seed(seed)
        np.random.seed(seed)
    
    def fit(self, x_train,targets_train, n_neurons):
        H = self.activation_hidden(torch.matmul(x_train.float(), self.weight.float()) + self.bias.float())
     #  H = self.activation_hidden(torch.matmul(x_train.float(), self.weight) + self.bias)
        print('output of hiddenlayer',H.shape)
        H_inv = torch.pinverse(H)

        beta = torch.matmul(H_inv, targets_train.float())

        self.beta = nn.Parameter(beta)
        self.output_weight = nn.Parameter(self.beta)
        print('output weight matrix',self.output_weight.shape)
        self.bias = nn.Parameter(torch.zeros((n_neurons,)))
        return self
    
    def encode(self, x):
       # h = self.activation_hidden(x.float() @ self.weight.t())
        print('input to encoder',x.shape)
        h=self.activation_hidden(torch.matmul(x.float(), self.weight.float()) + self.bias.float())
        return h
    
    def decode(self, h):
        x_pred = self.activation_output(h @ self.output_weight)
        return x_pred
    
    def clear_memory(self):
        if self.use_gpu and torch.cuda.is_available():
            torch.cuda.empty_cache()
        else:
            torch.cuda.empty_cache()

def reconstruction_error(model,data,targets_train):
    with torch.no_grad():
        encoded = model.encode(data)
        print('encoded.shape',encoded.shape)
        decoded = model.decode(encoded)
        print('decoded',decoded.shape)
        mse_loss = nn.MSELoss()(decoded, targets_train)
        cos_sim = cosine_similarity(targets_train.reshape(targets_train.shape[0], -1), decoded.reshape(decoded.shape[0], -1))
        cos_dis=1 - cos_sim.mean()
        return mse_loss.item(),cos_dis

def load_prompts(file_path):
    with open(file_path) as f:
        prompts = json.load(f)
    return prompts

def prepare_data(prompts):
    text_features = []
    for prompt in prompts:
        text_feature = np.array(prompt['word_embedding'])
        text_features.append(text_feature)
    return text_features

def prepare_targets(prompts):
    clip_embeddings = []
    for prompt in prompts:
        clip_embedding = np.array(prompt['clip_embedding'])
        clip_embeddings.append(clip_embedding)
    return clip_embeddings



prompts = load_prompts('./prompts_and_embeddings.json')

# Prepare data and targets
data = prepare_data(prompts)
targets = prepare_targets(prompts)
hidden_size=500
# Convert to PyTorch tensors


# Split the data into train and test sets
data_train, data_test, targets_train, targets_test = train_test_split(data, targets, test_size=0.2, random_state=42)

data_train=torch.tensor(data_train)
data_test=torch.tensor(data_test)
targets_train=torch.tensor(targets_train)
targets_test=torch.tensor(targets_test)
#print(targets_test.shape[1])
autoencoder = ELMAutoEncoder(data_train.shape[1], hidden_size,targets_test.shape[1])
autoencoder.fit(data_train,targets_train, hidden_size)
#print(data_train.shape)
train_reconstruction_error,cos_dis = reconstruction_error(autoencoder,data_train,targets_train)
# Calculate the mean squared error and cosine distance
print('train mse error','{:.15f}'.format(train_reconstruction_error))
print('train cosine distance','{:.15f}'.format(cos_dis))

test_reconstruction_error,cos_dis = reconstruction_error(autoencoder,data_test,targets_test)
print('test mse error',test_reconstruction_error)
print('test cosine distance',cos_dis)






<ipython-input-3-9a66e1d9b254>:114: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  data_train=torch.tensor(data_train)


output of hiddenlayer torch.Size([400, 500])
output weight matrix torch.Size([500, 768])
input to encoder torch.Size([400, 9216])
encoded.shape torch.Size([400, 500])
decoded torch.Size([400, 768])
train mse error 0.000007786394220
train cosine distance 0.403268322393927
input to encoder torch.Size([100, 9216])
encoded.shape torch.Size([100, 500])
decoded torch.Size([100, 768])
test mse error 0.00041739721020066125
test cosine distance 0.4564410068879795
